# Logging

First we need some imports.

In [111]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
import numpy as np
from scipy import stats
import pylab as plt
import torch
import torchist
import swyft

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

## Training data

Now we generate training data.  As simple example, we consider the model

$$
x = z + \epsilon
$$

where the parameter $z \sim \mathcal{N}(\mu = 0, \sigma = 1)$ is standard normal distributed, and $\epsilon \sim \mathcal{N}(\mu = 0, \sigma = 0.1)$ is a small noise contribution.  We are interested in the posterior of $z$ given a measurement of parameter $x$.

In [113]:
class Simulator(swyft.Simulator):
    def __init__(self):
        super().__init__()
        self.on_after_forward = swyft.to_numpy32

    def forward(self, trace):
        z = trace.sample('z', lambda: np.random.rand(1))
        x = trace.sample('x', lambda z: z + np.random.randn(1)*0.1, z)
        
sim = Simulator()

In [114]:
samples = sim.sample(10000)

100%|██████████| 10000/10000 [00:00<00:00, 46111.42it/s]


In [115]:
class Network(swyft.SwyftModule):
    def __init__(self):
        super().__init__(lr = 1e-4, lrs_factor = 0.1, lrs_patience = 3, early_stopping_patience = 2)
        self.logratios = swyft.LogRatioEstimator_1dim(num_features = 1, num_params = 1, varnames = 'z')

    def forward(self, A, B):
        logratios = self.logratios(A['x'], B['z'])
        return logratios

## Trainer

Training is now done using the `SwyftTrainer` class, which extends `pytorch_lightning.Trainer` by methods like `infer` (see below).

In [116]:

model_checkpoint = ModelCheckpoint(monitor = 'val_loss', save_top_k = 1)
logger = TensorBoardLogger("./lightning_logs", name = "Test1")

In [117]:
trainer = swyft.SwyftTrainer(accelerator = 'gpu', gpus=1, max_epochs = 100, precision = 64, logger = logger, callbacks = [model_checkpoint])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


The `swyft.Samples` class provides convenience functions to generate data loaders for training and validation data.

In [118]:
dl_train = samples[:2000].get_dataloader(batch_size = 64, shuffle = True)
dl_valid = samples[2000:5000].get_dataloader(batch_size = 64)

In [119]:
network = Network()

In [120]:
trainer.fit(network, dl_train, dl_valid)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | logratios | LogRatioEstimator_1dim | 17.4 K
-----------------------------------------------------
17.4 K    Trainable params
0         Non-trainable params
17.4 K    Total params
0.139     Total estimated model params size (MB)


Epoch 0:  41%|████      | 32/79 [00:00<00:00, 100.46it/s, loss=-0.151, v_num=21]
Validating: 0it [00:00, ?it/s]
Epoch 1:  41%|████      | 32/79 [00:00<00:00, 101.85it/s, loss=-0.401, v_num=21, val_loss=-.0107]
Validating: 0it [00:00, ?it/s]
Epoch 2:  41%|████      | 32/79 [00:00<00:00, 99.54it/s, loss=-0.522, v_num=21, val_loss=-.366] 
Validating: 0it [00:00, ?it/s]
Epoch 3:  41%|████      | 32/79 [00:00<00:00, 99.25it/s, loss=-0.557, v_num=21, val_loss=-.516] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  41%|████      | 32/79 [00:00<00:00, 88.67it/s, loss=-0.533, v_num=21, val_loss=-.527] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  41%|████      | 32/79 [00:00<00:00, 92.03it/s, loss=-0.597, v_num=21, val_loss=-.535] 
Validating: 0it [00:00, ?it/s]
Epoch 6:  41%|████      | 32/79 [00:00<00:00, 94.39it/s, loss=-0.507, v_num=21, val_loss=-.535] 
Validating: 0it [00:00, ?it/s]
Epoch 7:  41%|████      | 32/79 [00:00<00:00, 96.21it/s, loss=-0.578, v_num=21, val_loss=-.547] 
Validating: 0it [00:00

In [121]:
model_checkpoint.to_yaml("./test4.yaml")

In [122]:
ckpt_path = swyft.best_from_yaml("./test4.yaml")

In [123]:
trainer.test(network, dl_valid, ckpt_path = ckpt_path)

The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: EarlyStopping, LearningRateMonitor
Restoring states from the checkpoint path at ./lightning_logs/Test1/version_21/checkpoints/epoch=7-step=255.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ./lightning_logs/Test1/version_21/checkpoints/epoch=7-step=255.ckpt


Testing:  79%|███████▊  | 37/47 [00:00<00:00, 364.61it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': -0.5501115308490315}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 47/47 [00:00<00:00, 354.16it/s]


[{'test_loss': -0.5501115308490315}]